In [1]:
// The code was removed by DSX for sharing.

Waiting for a Spark session to start...

credentials = ......


com.ibm.ibmos2spark.CloudObjectStorage@1a08e5aa

In [2]:
import scala.collection.mutable.ListBuffer
import scala.util.control.Breaks.{break, breakable}
def CommunitiesAfterCut(vertices:Set[Int], cutmap:Map[Set[Int], Double]): Set[Set[Int]] ={
    var communities = Set.empty[Set[Int]]
    var samecom = Set.empty[Int]
    for (i <- vertices){
        breakable {
            if (samecom.contains(i)) {
                break()
            }
            else {
                var oldset = Set(i)
                var indicator = 0
                var con = cutmap.keys.filter(_.contains(i)).flatten.toSet.diff(oldset)
                oldset ++= con
                while (indicator != 1) {
                    var nextlv = Set.empty[Int]
                    for (j<-con) {
                        nextlv ++= cutmap.keys.filter(_.contains(j)).flatten.toSet.diff(oldset)
                    }
                    if (nextlv.size == 0){
                        indicator = 1
                    }
                    else{
                        con = nextlv
                        oldset ++= nextlv
                    }
                }
                communities += oldset
                samecom ++= vertices.intersect(oldset)
            }
        }
    }
    communities
}

CommunitiesAfterCut: (vertices: Set[Int], cutmap: Map[Set[Int],Double])Set[Set[Int]]


In [3]:
var raw = sc.textFile(cos.url("bufantest-donotdelete-pr-tkuiqj2r7rommw", "video_small_num.csv"))
var header = raw.first()
raw = raw.filter(row => row != header)
// user_prod list
val u_p_list = raw.map(row=>{
    var x = row.split(",")
    (x(0),x(1))
}).groupByKey().map(row=>{
    (row._1.toInt, row._2.toSet)
}).collect()

Waiting for a Spark session to start...

...


...

In [4]:
var edgemap = Map.empty[Set[Int], Double]
val nrows = u_p_list.length
// create graph
for (i<- 0 to nrows -1){
    for (j<-i+1 to nrows -1){
        var mutual = u_p_list(i)._2.intersect(u_p_list(j)._2)
        if (mutual.size >= 7){
            edgemap += Set(u_p_list(i)._1, u_p_list(j)._1) -> 0
        }
    }
}

edgemap = Map(Set(1206, 2694) -> 0.0, Set(780, 2005) -> 0.0, Set(2, 216) -> 0.0, Set(33, 65) -> 0.0, Set(2, 6) -> 0.0, Set(10, 30) -> 0.0, Set(1245, 640) -> 0.0, Set(50, 1303) -> 0.0, Set(3063, 2374) -> 0.0, Set(87, 69) -> 0.0, Set(473, 1) -> 0.0, Set(2696, 3268) -> 0.0, Set(660, 27) -> 0.0, Set(752, 1072) -> 0.0, Set(73, 16) -> 0.0, Set(4, 22) -> 0.0, Set(29, 1) -> 0.0, Set(3128, 1197) -> 0.0, Set(1620, 3280) -> 0.0, Set(65, 29) -> 0.0, Set(31, 1) -> 0.0, Set(1508, 780) -> 0.0, Set(871, 1648) -> 0.0, Set(590, 50) -> 0.0, Set(659, 661) -> 0.0, Set(915, 2694) -> 0.0, Set(475, 2575) -> 0.0, Set(752, 713) -> 0.0, Set(460, 87) -> 0.0, Set(1197, 915) -> 0.0, Set(1355, 780) -> 0.0, Set(275, 1) -> 0.0, Set(660, 5) -> 0.0, Set(4, 460) -> 0.0, Set...


Map(Set(1206, 2694) -> 0.0, Set(780, 2005) -> 0.0, Set(2, 216) -> 0.0, Set(33, 65) -> 0.0, Set(2, 6) -> 0.0, Set(10, 30) -> 0.0, Set(1245, 640) -> 0.0, Set(50, 1303) -> 0.0, Set(3063, 2374) -> 0.0, Set(87, 69) -> 0.0, Set(473, 1) -> 0.0, Set(2696, 3268) -> 0.0, Set(660, 27) -> 0.0, Set(752, 1072) -> 0.0, Set(73, 16) -> 0.0, Set(4, 22) -> 0.0, Set(29, 1) -> 0.0, Set(3128, 1197) -> 0.0, Set(1620, 3280) -> 0.0, Set(65, 29) -> 0.0, Set(31, 1) -> 0.0, Set(1508, 780) -> 0.0, Set(871, 1648) -> 0.0, Set(590, 50) -> 0.0, Set(659, 661) -> 0.0, Set(915, 2694) -> 0.0, Set(475, 2575) -> 0.0, Set(752, 713) -> 0.0, Set(460, 87) -> 0.0, Set(1197, 915) -> 0.0, Set(1355, 780) -> 0.0, Set(275, 1) -> 0.0, Set(660, 5) -> 0.0, Set(4, 460) -> 0.0, Set(19, 0) -> 0.0, Set(475, 713) -> 0.0, Set(31, 0) -> 0.0, Set(3, 322) -> 0.0, Set(1861, 1761) -> 0.0, Set(62, 20) -> 0.0, Set(1918, 915) -> 0.0, Set(125, 102) -> 0.0, Set(2694, 2696) -> 0.0, Set(1197, 2696) -> 0.0, Set(1498, 1648) -> 0.0, Set(228, 3) -> 0.0, Set(

In [5]:
var vertices = edgemap.flatMap(_._1.toList).toSet
//        println(edgemap.size)       // 498
//        println(vertices.size)      // 222
        // BFS with shortest path computation
var v_edge_map = Map.empty[Int, Set[Int]]
for (i<-vertices) {
    var shortpath = Map(i -> 1) // vertice -> int, stores shortest path
    var edge = Map.empty[Int, List[(Int, Int)]] // level -> List(beg, end), stores edge and it's level

    var level = 0
    var indicater = 0
    // setup first level
    var oldset = edgemap.keys.filter(_.contains(i)).toSet
    v_edge_map += (i -> oldset.flatten.diff(Set(i)))
    var firstlv = oldset.map(row => {
        (i, row.toList.filter(_ != i)(0))
    }).toList // List(i, xxxx)
    var oldsetsize = oldset.size
    edge += (0 -> firstlv)
    for (j <- firstlv) {
        shortpath += (j._2 -> 1)
    }
    // the lowest level
    var lowestlv = 0
    // BFS algorithm
    while (indicater != 1) {
        var nextlv = List.empty[(Int, Int)]
        for (x <- edge(level)) {
            var newconnections = edgemap.keys.filter(_.contains(x._2)).toSet.diff(oldset)
            var connected = newconnections.map(row => {
                (x._2, row.toList.filter(_ != x._2)(0))
            }) // (x, new connections)
            // filter out those connections at same level
            var validcon = connected.filter(row => {
                !(edge(level).map(_._2).contains(row._2))
            })
            // store the valid connections to next level
            nextlv = validcon.toList ::: nextlv
            oldset ++= newconnections
            // store shorest path
            for (k <- validcon) {
                if (shortpath.filterKeys(_ == k._2).isEmpty) {
                    shortpath += (k._2 -> shortpath(x._2))
                }
                else {
                    shortpath += (k._2 -> (shortpath(k._2) + shortpath(x._2)))
                }
            }
        }

        if (nextlv.isEmpty) {
            lowestlv = level
            indicater = 1
        }
        else {
            level += 1
            edge += (level -> nextlv)
        }
    }
    // now we have BFS tree, calculate the betweenness
    var btvalue = Map.empty[Int, Double] // vertices -> betweenness value, start with 1
    for (j <- shortpath.keys) {
        btvalue += (j -> 1)
    }
    //            edge(3).filter(_._2 == 1761).map(row=>{(row, shortpath(row._1))}).foreach(println)
    // loop backwards from bottom to top
    for (l <- lowestlv to 0 by -1) {
        // loop each vertices and add its edges' betweenness to the edgemap
        for (m <- edge(l).map(_._2).distinct) {
            // find edges that contains m
            var cand = edge(l).filter(_._2 == m) //.map(row=>{(Set(row._1, row._2), shortpath(row._1))})
            // distribute betweenness value to edges and add betweenness value to higher nodes
            val spath = shortpath(m)
            for (n <- cand) {
                val bt = 1.0 * btvalue(m) * shortpath(n._1) / spath
                edgemap += (Set(n._1, n._2) -> (edgemap(Set(n._1, n._2)) + bt))
                btvalue += (n._1 -> (btvalue(n._1) + bt))
            }

        }
    }
}

vertices = Set(69, 2744, 2876, 909, 468, 0, 2976, 762, 217, 2953, 5, 120, 10, 142, 1761, 2901, 2575, 42, 2985, 24, 25, 257, 52, 14, 184, 372, 125, 3280, 1992, 196, 460, 20, 284, 228, 78, 3268, 29, 216, 475, 321, 1418, 253, 485, 1136, 2281, 1072, 774, 660, 1, 265, 74, 307, 1206, 6, 60, 661, 220, 366, 102, 1723, 1355, 28, 38, 160, 192, 2606, 275, 21, 2150, 2414, 33, 65, 2332, 697, 1270, 2801, 1794, 1121, 1111, 1498, 2694, 3102, 109, 328, 3000, 590, 706, 1150, 1530, 1231, 2862, 41, 605, 134, 73, 105, 2, 541, 1648, 166, 34, 45, 149, 22, 1615, 1389, 44, 2188, 1620, 59, 118, 1256, 27, 71, 12, 382, 54, 713, 672, 113, 681, 1408, 274, 2097, 2955, 7, 245, 2116, 39, 208, 1667, 103, 213, 483, 66, 1325, 915, 108, 330, 1245, 780, 2288, 3, 241, 35, 2005, 473, 209, ...


Set(69, 2744, 2876, 909, 468, 0, 2976, 762, 217, 2953, 5, 120, 10, 142, 1761, 2901, 2575, 42, 2985, 24, 25, 257, 52, 14, 184, 372, 125, 3280, 1992, 196, 460, 20, 284, 228, 78, 3268, 29, 216, 475, 321, 1418, 253, 485, 1136, 2281, 1072, 774, 660, 1, 265, 74, 307, 1206, 6, 60, 661, 220, 366, 102, 1723, 1355, 28, 38, 160, 192, 2606, 275, 21, 2150, 2414, 33, 65, 2332, 697, 1270, 2801, 1794, 1121, 1111, 1498, 2694, 3102, 109, 328, 3000, 590, 706, 1150, 1530, 1231, 2862, 41, 605, 134, 73, 105, 2, 541, 1648, 166, 34, 45, 149, 22, 1615, 1389, 44, 2188, 1620, 59, 118, 1256, 27, 71, 12, 382, 54, 713, 672, 113, 681, 1408, 274, 2097, 2955, 7, 245, 2116, 39, 208, 1667, 103, 213, 483, 66, 1325, 915, 108, 330, 1245, 780, 2288, 3, 241, 35, 2005, 473, 209, 355, 194, 145, 525, 640, 18, 414, 95, 739, 50, 3063, 16, 463, 576, 359, 31, 2113, 656, 2543, 175, 386, 446, 497, 43, 1918, 2479, 2696, 87, 1913, 688, 2997, 1866, 26, 1508, 114, 8, 1861, 75, 1030, 119, 58, 2739, 82, 2374, 752, 2517, 36, 146, 30, 1303, 

In [6]:
val ranks = edgemap.map(_._2).toList.distinct.sorted.reverse        // 271 distinct values
// cut from top to bottom, store in a Map(modularity -> community)
val m = edgemap.size
var cut = 0
var modmap = Map.empty[(Int, Double), Set[Set[Int]]]
for (i <- ranks) {
//            println(cut)
    var mod = 0.0
    // find edges with betweenness i in edge map
    var tocut = edgemap.filter(_._2 >= i).keys
    for (n <- tocut){
        var tmp = n.toList
        v_edge_map += (tmp(0) -> v_edge_map(tmp(0)).diff(Set(tmp(1))))
        v_edge_map += (tmp(1) -> v_edge_map(tmp(1)).diff(Set(tmp(0))))
    }
    // remove these edges from edgemap
    var tmpmap = edgemap
    tmpmap --= tocut
    // BFS again on the cut nodes to detect separated communities
    var communities = CommunitiesAfterCut(vertices, tmpmap)
    // for each community calculate the modularity and then sum them up
    for (j <- communities) {
        val tmp = j.toList
            for (k <- 0 to (tmp.length - 1)){
                var ki = v_edge_map(tmp(k)).size
                for (l <- (k+1 to (tmp.length - 1))){
                    var Aij = 0.0
                    if (v_edge_map(tmp(k)).contains(tmp(l))) {
                        Aij = 1.0
                    }
                    var kj = v_edge_map(tmp(l)).size
                    mod += Aij - 0.5 * ki * kj / m
                }
            }

    }
    cut += 1
    modmap += ((cut, (0.5 * mod / m)) -> communities)

}

ranks = List(8468.0, 7972.974603174607, 7714.0, 7448.0, 7339.025396825394, 6670.0, 6578.0, 4433.234942602823, 3938.0, 3796.0, 3723.9999999999995, 2801.76283845689, 2616.229042720459, 2140.7439782439774, 1560.0, 1529.4999999999998, 1486.4981306500745, 1289.1802671455662, 1276.1351939854244, 1249.0, 1225.3965589965592, 1224.5, 1217.24493261281, 1164.1965589965598, 1122.0, 1108.900895363509, 1059.494538562416, 1000.4285714285716, 934.2666666666668, 814.6403170830798, 781.3764013764013, 775.7674603174592, 758.207984975336, 754.9936727615508, 752.0, 744.0, 726.3333333333334, 688.9333333333333, 680.0, 661.1436727615505, 653.1837419618464, 560.0, 556.0, 530.176372143723, 495.07658627921705, 491.6158286158291, 485.9999999999999, 476.1428571428571, 431.26666666666665, 406.356021756...


List(8468.0, 7972.974603174607, 7714.0, 7448.0, 7339.025396825394, 6670.0, 6578.0, 4433.234942602823, 3938.0, 3796.0, 3723.9999999999995, 2801.76283845689, 2616.229042720459, 2140.7439782439774, 1560.0, 1529.4999999999998, 1486.4981306500745, 1289.1802671455662, 1276.1351939854244, 1249.0, 1225.3965589965592, 1224.5, 1217.24493261281, 1164.1965589965598, 1122.0, 1108.900895363509, 1059.494538562416, 1000.4285714285716, 934.2666666666668, 814.6403170830798, 781.3764013764013, 775.7674603174592, 758.207984975336, 754.9936727615508, 752.0, 744.0, 726.3333333333334, 688.9333333333333, 680.0, 661.1436727615505, 653.1837419618464, 560.0, 556.0, 530.176372143723, 495.07658627921705, 491.6158286158291, 485.9999999999999, 476.1428571428571, 431.26666666666665, 406.3560217560217, 378.00000000000006, 378.0, 376.0, 374.00000000000006, 373.00000000000006, 373.0, 372.66666666666663, 372.00000000000006, 372.0, 371.6666666666667, 371.3333333333333, 368.66666666666663, 368.0, 364.3333333333333, 358.333

In [13]:
val out = modmap(modmap.keys.maxBy(_._2)).toList.map(row=>{row.toList.sorted}).sortBy(row => {row(0)})

out = List(List(0, 1, 3, 4, 5, 7, 10, 12, 14, 16, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 31, 33, 34, 36, 38, 39, 41, 42, 43, 44, 50, 52, 59, 60, 62, 65, 66, 68, 69, 71, 73, 74, 75, 78, 82, 83, 87, 90, 95, 103, 105, 108, 109, 113, 114, 118, 120, 134, 145, 146, 149, 160, 175, 183, 184, 192, 194, 196, 208, 209, 213, 217, 220, 227, 228, 241, 253, 254, 257, 265, 274, 275, 284, 307, 321, 322, 328, 330, 355, 366, 372, 382, 386, 414, 446, 460, 463, 473, 483, 485, 525, 532, 590, 605, 644, 656, 660, 672, 688, 697, 706, 723, 728, 739, 762, 774, 1030, 1111, 1121, 1150, 1256, 1303, 1325, 1389, 1667, 2288), List(2, 6, 35, 216), List(8, 119, 1615, 2543), List(30), List(45, 277), List(54, 102, 125), List(58, 166, 245), List(111, 681), List(142, 1231), List(283, 2281), List(359, 46...


List(List(0, 1, 3, 4, 5, 7, 10, 12, 14, 16, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 31, 33, 34, 36, 38, 39, 41, 42, 43, 44, 50, 52, 59, 60, 62, 65, 66, 68, 69, 71, 73, 74, 75, 78, 82, 83, 87, 90, 95, 103, 105, 108, 109, 113, 114, 118, 120, 134, 145, 146, 149, 160, 175, 183, 184, 192, 194, 196, 208, 209, 213, 217, 220, 227, 228, 241, 253, 254, 257, 265, 274, 275, 284, 307, 321, 322, 328, 330, 355, 366, 372, 382, 386, 414, 446, 460, 463, 473, 483, 485, 525, 532, 590, 605, 644, 656, 660, 672, 688, 697, 706, 723, 728, 739, 762, 774, 1030, 1111, 1121, 1150, 1256, 1303, 1325, 1389, 1667, 2288), List(2, 6, 35, 216), List(8, 119, 1615, 2543), List(30), List(45, 277), List(54, 102, 125), List(58, 166, 245), List(111, 681), List(142, 1231), List(283, 2281), List(359, 468), List(475, 713, 752, 1072, 1270, 1565, 1620, 2479, 2575, 2976, 3280), List(497, 935, 1530, 1992, 2116), List(541, 780, 871, 909, 915, 1136, 1197, 1206, 1355, 1408, 1418, 1498, 1508, 1648, 1723, 1761, 1861, 1913, 1918, 2005,